# Step 1: Import libraries and load CSVs

In [1]:
import pandas as pd
import os
from docx import Document
from PIL import Image
import pytesseract

# Load CSVs
train_csv = pd.read_csv("/Users/arshinawaz/Desktop/ML project/ass 1 data/train.csv")
test_csv = pd.read_csv("/Users/arshinawaz/Desktop/ML project/ass 1 data/test.csv")

# Show first few rows
train_csv.head()


,File Name,Aggrement Value,Aggrement Start Date,Aggrement End Date,Renewal Notice (Days),Party One,Party Two
0,6683127-House-Rental-Contract-GERALDINE-GALINA...,6500,20.05.2007,20.05.2008,15.0,"Antonio Levy S. Ingles, Jr. and/or Mary Rose C...",GERALDINE Q. GALINATO
1,6683129-House-Rental-Contract-Geraldine-Galina...,6500,20.05.2007,20.05.2008,15.0,"Antonio Levy S. Ingles, Jr. and/or Mary Rose C...",GERALDINE Q. GALINATO
2,18325926-Rental-Agreement-1,4000,05.12.2008,31.11.2009,90.0,MR.K.Kuttan,P.M. Narayana Namboodri
3,24158401-Rental-Agreement,12000,01.04.2008,31.03.2009,60.0,Hanumaiah,Vishal Bhardwaj
4,36199312-Rental-Agreement,3800,01.05.2010,31.04.2011,30.0,Balaji.R,Kartheek R


# Step 2: Define functions to read documents


# Read all files from the folder

In [6]:
import os
from docx import Document
from PIL import Image
import pytesseract

# Define folder paths
train_folder = "/Users/arshinawaz/Desktop/ML project/ass 1 data/train"
test_folder = "/Users/arshinawaz/Desktop/ML project/ass 1 data/test"

# Functions to read documents
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def read_image(file_path):
    img = Image.open(file_path)
    return pytesseract.image_to_string(img)

def read_all_files(folder_path):
    texts = {}
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if filename.endswith(".docx"):
                texts[filename] = read_docx(file_path)
            elif filename.endswith(".png") or filename.endswith(".jpg"):
                texts[filename] = read_image(file_path)
        except:
            print(f"Failed to read {filename}")
    return texts

# Read train and test documents
train_texts = read_all_files(train_folder)
test_texts = read_all_files(test_folder)


In [17]:
# Drop rows where text is NaN
train_csv = train_csv.dropna(subset=['text'])

# Features and labels for Agreement Value
X = train_csv['text']
y = train_csv['Aggrement Value']


In [18]:
# Merge text with CSV labels”
import pandas as pd

# Load CSVs
train_csv = pd.read_csv("/Users/arshinawaz/Desktop/ML project/ass 1 data/train.csv")
test_csv = pd.read_csv("/Users/arshinawaz/Desktop/ML project/ass 1 data/test.csv")

# Merge document texts into CSV using the filename
train_csv['text'] = train_csv['File Name'].map(train_texts)
test_csv['text'] = test_csv['File Name'].map(test_texts)

# Check the first few rows
print(train_csv.head())


                                           File Name  Aggrement Value  \
0  6683127-House-Rental-Contract-GERALDINE-GALINA...             6500   
1  6683129-House-Rental-Contract-Geraldine-Galina...             6500   
2                        18325926-Rental-Agreement-1             4000   
3                          24158401-Rental-Agreement            12000   
4                          36199312-Rental-Agreement             3800   

  Aggrement Start Date Aggrement End Date  Renewal Notice (Days)  \
0           20.05.2007         20.05.2008                   15.0   
1           20.05.2007         20.05.2008                   15.0   
2           05.12.2008         31.11.2009                   90.0   
3           01.04.2008         31.03.2009                   60.0   
4           01.05.2010         31.04.2011                   30.0   

                                           Party One  \
0  Antonio Levy S. Ingles, Jr. and/or Mary Rose C...   
1  Antonio Levy S. Ingles, Jr. and/or Ma

In [19]:
import os

# Fix dictionary keys
train_texts_fixed = {os.path.splitext(k)[0].lower().strip(): v for k, v in train_texts.items()}
test_texts_fixed = {os.path.splitext(k)[0].lower().strip(): v for k, v in test_texts.items()}

# Map to CSV
train_csv['text'] = train_csv['File Name'].str.lower().str.strip().map(train_texts_fixed)
test_csv['text'] = test_csv['File Name'].str.lower().str.strip().map(test_texts_fixed)


In [20]:
print(train_csv[['File Name', 'text']].head())
print(train_csv['text'].isna().sum())  # Should be 0


                                           File Name  \
0  6683127-House-Rental-Contract-GERALDINE-GALINA...   
1  6683129-House-Rental-Contract-Geraldine-Galina...   
2                        18325926-Rental-Agreement-1   
3                          24158401-Rental-Agreement   
4                          36199312-Rental-Agreement   

                                                text  
0  House Rental Contract\nKNOWN ALL MEN BY THESE ...  
1  \n\n\n\n\nHouse Rental Contract\nKNOWN ALL MEN...  
2  \n\n\n\n\n\n\n\n\n\nRENTAL AGREEMENT\nThis dee...  
3                                                NaN  
4  RENEWAL OF RENTAL AGREEMENT\n\nThis AGREEMENT ...  
1


# Train a Simple Model

In [12]:
# Step 1: Import ML libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [14]:
# Step 1: Features (X) and label (y) for Agreement Value
X = train_csv['text']                   # Text of the documents
y = train_csv['Aggrement Value']        # The metadata field you want to predict

In [24]:
# prepare the data 

# Drop rows with missing text
train_csv = train_csv.dropna(subset=['text'])


# Features and labels for one example field: Agreement Value
X = train_csv['text']
y = train_csv['Aggrement Value']  # you can repeat for other fields later


In [23]:
# step 3 split the data 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_val_vec)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy for Agreement Value:", accuracy)


Validation Accuracy for Agreement Value: 0.5
